<a href="https://colab.research.google.com/github/bcastiblancoo/Physics-Analysis-Using-ATLAS-Open-Data-Releases/blob/main/(PANDAS)-HWW_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**HWW Analysis From ATLAS Open Data 13 TeV Release**

Import and install necessary extensions:

In [1]:
import sys
!pip install uproot3 vector numpy matplotlib #awkward

     |████████████████████████████████| 117 kB 12.3 MB/s 
     |████████████████████████████████| 155 kB 75.7 MB/s 
     |████████████████████████████████| 87 kB 7.1 MB/s 


Import some tools which will be useful in analysis, and the data set with MC information:

In [2]:
import uproot3 # library for reading .root files
import pandas as pd # to represent nested data in columnar format
#import awkward as ak
import vector # for 4-momentum calculations
import time # to measure time to analyse
import math # for mathematical functions such as square root
import numpy as np # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import AutoMinorLocator # for minor ticks

import datasetfile # local file containing all dataset IDs, number of events, filter, cross-sections and sums of weights

**DATA**
Access to input files used in this analysis and set the luminosity (fraction) of the DATA we want to analyze:

In [3]:
L=10 #fb^{-1}
fraction = 1.0 # reduce this is if you want the code to run quicker
tuple_path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/" # web address to selected data (Two-Lepton Final State)

Now, define the samples to process (**DATA and MC**):

In [4]:
samples = {

    'data': {
        'list' : ['data_A','data_B','data_C','data_D'],
    },

    r'Single top' : { # t-----> 2lep + jets
        'list' : ['single_top_tchan','single_antitop_tchan','single_top_schan','single_antitop_schan','single_top_wtchan','single_antitop_wtchan'],
        'color' : "#046865" # skobeloff
    },

    r't\bar{t}$' : { #ttbar ------> 2b-jets + 2lep
        'list' : ['ttbar_lep'],
        'color' : "#6b59d3" # slate blue
    },

    r'V + jets' : { # W + jets & Z + jets ---> 2lep +sth
        'list' : ['Zmumu','Ztautau','Wplusenu','Wplusmunu','Wplustaunu','Wminusenu','Wminusmunu','Wminustaunu','Zee'],
        'color' : "#ffe900" # middle yellow
    },

      r'Diboson $WW$' : { # WW
        'list' : ['WqqZll','WpqqWmlv','WplvWmqq','WlvZqq','llvv','lvvv','llll', 'ZqqZll', 'lllv'], 
        'color' : "#ff0000" # red
    },

      r'Higgs ($m_H$ = 125 GeV)' : { # H ----> WW -----> e-nu_e-mu-nu_mu
        'list' : ['ggH125_WW2lep','VBFH125_WW2lep','WH125_ZZ4lep','ZH125_ZZ4lep'],
        'color' : "#00cdff" # vivid sky blue
    },

    
}

Let's define the units, taking into account that data is in GeV by default:

In [6]:
GeV=1.0
MeV=0.001

Define function to get data from files.

The datasets used in this notebook have already been filtered to include at least 2 leptons per event, so that processing is quicker.

In [5]:
def get_data_from_files():

    data = {} # define empty dictionary to hold awkward arrays
    for s in samples: # loop over samples
        print('Processing '+s+' samples') # print which sample
        frames = [] # define empty list to hold data
        for val in samples[s]['list']: # loop over each file
            if s == 'data': prefix = "Data/" # Data prefix
            else: # MC prefix
                prefix = "MC/mc_"+str(datasetfile.infos[val]["DSID"])+"."
            fileString = tuple_path+prefix+val+".2lep.root" # file name to open
            print('filestring:',fileString)
            temp = read_file(fileString,val) # call the function read_file defined below
            frames.append(temp) # append dataframe returned from read_file to list of dataframes
        data[s] = pd.concat(frames) # dictionary entry is concatenated awkward arrays
    
    return data # return dictionary of awkward arrays

define function to calculate weight of MC event

In [6]:
def calc_weight(xsec_weight, events):
    return (
        xsec_weight
        * events.mcWeight
        * events.scaleFactor_PILEUP
        * events.scaleFactor_ELE
        * events.scaleFactor_MUON 
        * events.scaleFactor_LepTRIGGER
    )

define function to get cross-section weight

In [7]:
def get_xsec_weight(sample):
    info = datasetfile.infos[sample] # open infofile
    xsec_weight = (L*1000*info["xsec"])/(info["sumw"]*info["red_eff"]) #*1000 to go from fb-1 to pb-1
    return xsec_weight # return cross-section weight

define function to calculate 2-lepton invariant mass.

Note: lep_(pt|eta|phi|E) are variable length lists of lepton momentum components for each event, represented by awkward arrays.

In [ ]:
#def calc_mT_ll(lep_pt, lep_eta, lep_phi, lep_E):
    # construct awkward 4-vector array
#    pT = vector.awk(ak.zip(dict(pt=lep_pt, eta=lep_eta, phi=lep_phi, E=lep_E)))
    # calculate invariant mass of the first 2 leptons 
    # [:, i] selects the i-th lepton in each event
    # .M calculates the invariant mass
#    return (pT[:, 0] + pT[:, 1]).M * MeV

In [ ]:
#def calc_MET(met_pt, met_eta, met_phi, met_E):
    # construct awkward 4-vector array
#    MET = vector.awk(ak.zip(dict(pt=met_pt, eta=met_eta, phi=met_phi, E=met_E)))
    # calculate invariant mass of the first MET 
    # [:, i] selects the i-th lepton in each event
    # .M calculates the invariant mass
#    return (MET[] + MET[]).M * MeV

In [ ]:
#def calc_mT(mT_pt, mT_eta, mT_phi, mT_E):
#  mT = vector.awk(ak.zip(dict(pt=mT_pt, eta=mT_eta, phi=mT_phi, E=mT_E)))
#  return (mT[]).M * MeV

Changing a cut

If you change a cut: Cell -> Run All Below

If you change a cut here, you also need to make sure the cut is applied in the "Applying a cut" cell.

In [8]:
# cut on lepton charge
# paper: "Exactly two isolated, different-flavour opposite-sign leptons (electrons or muons)"
def cut_lep_charge(lep_charge):
# throw away when sum of lepton charges is not equal to 0
    return lep_charge[:, 0] + lep_charge[:, 1] != 0

# cut on lepton type
# paper: "Exactly two isolated, different-flavour opposite-sign leptons (electrons or muons)"
def cut_lep_type(lep_type):
# for an electron lep_type is 11
# for a muon lep_type is 13
# throw away when none of emu
    sum_lep_type = lep_type[:, 0] + lep_type[:, 1]
    return (sum_lep_type != 24)

Applying a cut

If you add a cut: Cell -> Run All Below

In [13]:
def read_file(path,sample):
    start = time.time() # start the clock
    print("\tProcessing: "+sample) # print which sample is being processed
    data_all = pd.DataFrame() # define empty pandas DataFrame to hold all data for this sample
    tree = uproot3.open(path)["mini"] # open the tree called mini
    numevents = uproot3.numentries(path, "mini") # number of events
    if 'data' not in sample: xsec_weight = get_xsec_weight(sample) # get cross-section weight
    for data in tree.iterate(['lep_pt','lep_eta','lep_phi',
                              'lep_E','lep_charge','lep_type',
                              'met_et','met_phi',
                              # add more variables here if you make cuts on them 
                              'mcWeight','scaleFactor_PILEUP',
                              'scaleFactor_ELE','scaleFactor_MUON',
                              'scaleFactor_LepTRIGGER'], # variables to calculate Monte Carlo weight
                             outputtype = pd.DataFrame, # choose output type as pandas DataFra
                             entry_stop=numevents*fraction): # process up to numevents*fraction
                             
                            
            nIn = len(data) # number of events in this batch

            if 'data' not in sample: # only do this for Monte Carlo simulation files
                # multiply all Monte Carlo weights and scale factors together to give total weight
                data['totalWeight'] = calc_weight(xsec_weight, data)

            # cut on lepton charge using the function cut_lep_charge defined above
            data = data[~cut_lep_charge(data.lep_charge)]

            # cut on lepton type using the function cut_lep_type defined above
            data = data[~cut_lep_type(data.lep_type)]
            
            # construct awkward 4-vectors array for the pair of leptons and the MET:

            #p2 = vector.awk(ak.zip(dict(pt=data.lep_pt, eta=data.lep_eta, phi=data.lep_phi, E=data.lep_E)))
            #met= vector.awk(ak.zip(dict(pt=data.met_et, eta=0, phi=data.met_phi, E=data.met_et)))

            #With numpay:
            lispt1=[data.lep_pt[0],data.lep_eta[0],data.lep_phi[0],data.lep_E[0]]
            lispt2=[data.lep_pt[1],data.lep_eta[1],data.lep_phi[1],data.lep_E[1]]
            pl1=np.array(lispt1)
            pl2=np.array(lispt2)
            p2=pl1+pl2
            el1=data.lep_E[0]
            el2=data.lep_E[1]
            e2=el1+el2

            liset=[data.met_et,0,data.met_phi,data.met_et]
            pmet=np.array(liset)
            met=data.met_et

            # calculate invariant mass of first 2 leptons + met
            # [:, i] selects the i-th lepton in each event
            # .M calculates the invariant mass
            mT=math.sqrt((e2+met)**2-abs(p2+pmet))* MeV

            # calculation of 2-lepton +MET invariant mass using the function calc_mT defined above
            data['mT'] = mT

            # array contents can be printed at any stage like this
            #print(data)

            # array column can be printed at any stage like this
            #print(data['lep_pt'])

            # multiple array columns can be printed at any stage like this
            #print(data[['lep_pt','lep_eta']])

            nOut = len(data) # number of events passing cuts in this batch
            data_all = data_all.append(data) # append dataframe from this batch to the dataframe for the whole sample
            elapsed = time.time() - start # time taken to process
            print("\t\t nIn: "+str(nIn)+",\t nOut: \t"+str(nOut)+"\t in "+str(round(elapsed,1))+"s") # events before and after
    
    return data_all # return dataframe containing events passing all cuts

This is where the processing happens

In [14]:
start = time.time() # time at start of whole processing
data = get_data_from_files() # process all files
elapsed = time.time() - start # time after whole processing
print("Time taken: "+str(round(elapsed,1))+"s") # print total time taken to process every file

Processing data samples
filestring: https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/Data/data_A.2lep.root
	Processing: data_A


TypeError: ignored

**Plotting**

If you only want to make a change in plotting: Cell -> Run All Below

Define function to plot the data

In [ ]:
def plot_data(data):

    xmin = 50 * GeV
    xmax = 210 * GeV
    step_size = 5 * GeV

    bin_edges = np.arange(start=xmin, # The interval includes this value
                     stop=xmax+step_size, # The interval doesn't include this value
                     step=step_size ) # Spacing between values
    bin_centres = np.arange(start=xmin+step_size/2, # The interval includes this value
                            stop=xmax+step_size/2, # The interval doesn't include this value
                            step=step_size ) # Spacing between values

    data_x,_ = np.histogram(pd.DataFrame(data['data']['mT']).to_numpy()), 
                            bins=bin_edges ) # histogram the data
    data_x_errors = np.sqrt( data_x ) # statistical error on the data

    signal_x = pd.DataFrame(data[r'Higgs ($m_H$ = 125 GeV)']['mT']).to_numpy() # histogram the signal
    signal_weights = pd.DataFrame(data[r'Higgs ($m_H$ = 125 GeV)'].totalWeight).to_numpy() # get the weights of the signal events
    signal_color = samples[r'Higgs ($m_H$ = 125 GeV)']['color'] # get the colour for the signal bar

    mc_x = [] # define list to hold the Monte Carlo histogram entries
    mc_weights = [] # define list to hold the Monte Carlo weights
    mc_colors = [] # define list to hold the colors of the Monte Carlo bars
    mc_labels = [] # define list to hold the legend labels of the Monte Carlo bars

    for s in samples: # loop over samples
        if s not in ['data', r'Higgs ($m_H$ = 125 GeV)']: # if not data nor signal
            mc_x.append( pd.DataFrame(data[s]['mT']).tp_numpy() ) # append to the list of Monte Carlo histogram entries
            mc_weights.append( pd.DataFrame(data[s].totalWeight).to_numpy() ) # append to the list of Monte Carlo weights
            mc_colors.append( samples[s]['color'] ) # append to the list of Monte Carlo bar colors
            mc_labels.append( s ) # append to the list of Monte Carlo legend labels
    


    # *************
    # Main plot 
    # *************
    main_axes = plt.gca() # get current axes
    
    # plot the data points
    main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors,
                       fmt='ko', # 'k' means black and 'o' is for circles 
                       label='Data') 
    
    # plot the Monte Carlo bars
    mc_heights = main_axes.hist(mc_x, bins=bin_edges, 
                                weights=mc_weights, stacked=True, 
                                color=mc_colors, label=mc_labels )
    
    mc_x_tot = mc_heights[0][-1] # stacked background MC y-axis value
    
    # calculate MC statistical uncertainty: sqrt(sum w^2)
    mc_x_err = np.sqrt(np.histogram(np.hstack(mc_x), bins=bin_edges, weights=np.hstack(mc_weights)**2)[0])
    
    # plot the signal bar
    main_axes.hist(signal_x, bins=bin_edges, bottom=mc_x_tot, 
                   weights=signal_weights, color=signal_color,
                   label=r'Higgs ($m_H$ = 125 GeV)')
    
    # plot the statistical uncertainty
    main_axes.bar(bin_centres, # x
                  2*mc_x_err, # heights
                  alpha=0.5, # half transparency
                  bottom=mc_x_tot-mc_x_err, color='none', 
                  hatch="////", width=step_size, label='Stat. Unc.' )

    # set the x-limit of the main axes
    main_axes.set_xlim( left=xmin, right=xmax ) 
    
    # separation of x axis minor ticks
    main_axes.xaxis.set_minor_locator( AutoMinorLocator() ) 
    
    # set the axis tick parameters for the main axes
    main_axes.tick_params(which='both', # ticks on both x and y axes
                          direction='in', # Put ticks inside and outside the axes
                          top=True, # draw ticks on the top axis
                          right=True ) # draw ticks on right axis
    
    # x-axis label
    main_axes.set_xlabel(r'2-lepton invariant mass $m_{T}$ [GeV]',
                        fontsize=13, x=1, horizontalalignment='right' )
    
    # write y-axis label for main axes
    main_axes.set_ylabel('Events / '+str(step_size)+' GeV',
                         y=1, horizontalalignment='right') 
    
    # set y-axis limits for main axes
    main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.6 )
    
    # add minor ticks on y-axis for main axes
    main_axes.yaxis.set_minor_locator( AutoMinorLocator() ) 

    # Add text 'ATLAS Open Data' on plot
    plt.text(0.05, # x
             0.93, # y
             'ATLAS Open Data', # text
             transform=main_axes.transAxes, # coordinate system used is that of main_axes
             fontsize=13 ) 
    
    # Add text 'for education' on plot
    plt.text(0.05, # x
            0.76, # y
             'Reproduced by: Brayan E. Castiblanco O.', # text
             transform=main_axes.transAxes, # coordinate system used is that of main_axes
             style='italic',
             fontsize=8 ) 
    
    # Add energy and luminosity
    lumi_used = str(L*fraction) # luminosity to write on the plot
    plt.text(0.05, # x
             0.88, # y
             '$\sqrt{s}$=13 TeV,$\int$L dt = '+lumi_used+' fb$^{-1}$', # text
             transform=main_axes.transAxes ) # coordinate system used is that of main_axes
    
    # Add a label for the analysis carried out
    plt.text(0.05, # x
             0.82, # y
             r'$H \rightarrow WW^* \rightarrow e\nu \mu\nu$', # text 
             transform=main_axes.transAxes ) # coordinate system used is that of main_axes

    # draw the legend
    main_axes.legend( frameon=False ) # no box around the legend
    
    return

Call the function to plot the data

In [ ]:
plot_data(data)